# Sampling

A recurrence Neural Network can be used as a Generative model once it was trained. Currently this is a common practice not only to study how well a model has learned a problem, but to learn more about the problem domain itself. In fact, this approach is being used for music generation and composition.

The process of generation is explained in the picture below:

<img src="Images/dinos3.png" style="width:500;height:300px;">
<caption><center> **Figure **: In this picture, we assume the model is already trained. We pass in $x^{\langle 1\rangle} = \vec{0}$ at the first time step, and have the network then sample one character at a time. </center></caption>

Let's do an example:

In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense, Masking
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /home/julian/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
# load ascii text and covert to lowercase
raw_text = nltk.corpus.gutenberg.raw('bible-kjv.txt')

We are going to traing a LSTM network using some passges from the bible, and we are going to use the trained netwrok to generate new "Gospels".

In [4]:
raw_text[100:1000]

u'Genesis\n\n\n1:1 In the beginning God created the heaven and the earth.\n\n1:2 And the earth was without form, and void; and darkness was upon\nthe face of the deep. And the Spirit of God moved upon the face of the\nwaters.\n\n1:3 And God said, Let there be light: and there was light.\n\n1:4 And God saw the light, that it was good: and God divided the light\nfrom the darkness.\n\n1:5 And God called the light Day, and the darkness he called Night.\nAnd the evening and the morning were the first day.\n\n1:6 And God said, Let there be a firmament in the midst of the waters,\nand let it divide the waters from the waters.\n\n1:7 And God made the firmament, and divided the waters which were\nunder the firmament from the waters which were above the firmament:\nand it was so.\n\n1:8 And God called the firmament Heaven. And the evening and the\nmorning were the second day.\n\n1:9 And God said, Let the waters under the heav'

In [5]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
char_to_int

{u'\n': 0,
 u' ': 1,
 u'!': 2,
 u"'": 3,
 u'(': 4,
 u')': 5,
 u',': 6,
 u'-': 7,
 u'.': 8,
 u'0': 9,
 u'1': 10,
 u'2': 11,
 u'3': 12,
 u'4': 13,
 u'5': 14,
 u'6': 15,
 u'7': 16,
 u'8': 17,
 u'9': 18,
 u':': 19,
 u';': 20,
 u'?': 21,
 u'A': 22,
 u'B': 23,
 u'C': 24,
 u'D': 25,
 u'E': 26,
 u'F': 27,
 u'G': 28,
 u'H': 29,
 u'I': 30,
 u'J': 31,
 u'K': 32,
 u'L': 33,
 u'M': 34,
 u'N': 35,
 u'O': 36,
 u'P': 37,
 u'Q': 38,
 u'R': 39,
 u'S': 40,
 u'T': 41,
 u'U': 42,
 u'V': 43,
 u'W': 44,
 u'Y': 45,
 u'Z': 46,
 u'[': 47,
 u']': 48,
 u'a': 49,
 u'b': 50,
 u'c': 51,
 u'd': 52,
 u'e': 53,
 u'f': 54,
 u'g': 55,
 u'h': 56,
 u'i': 57,
 u'j': 58,
 u'k': 59,
 u'l': 60,
 u'm': 61,
 u'n': 62,
 u'o': 63,
 u'p': 64,
 u'q': 65,
 u'r': 66,
 u's': 67,
 u't': 68,
 u'u': 69,
 u'v': 70,
 u'w': 71,
 u'x': 72,
 u'y': 73,
 u'z': 74}

In [7]:
n_chars = len(raw_text)
n_vocab = len(chars)
print "Total Characters: ", n_chars
print "Total Vocab: ", n_vocab

Total Characters:  4332554
Total Vocab:  75


To train the model we are going to use sequences of 100 characters

In [8]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print "Total Patterns: ", n_patterns

Total Patterns:  4332454


In [9]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [10]:
X.shape

(4332454, 100, 1)

Because of the data set is to large, we are going to use only the firs 200000 sequences.

In [11]:
X = X[:200000,:,:]

In [12]:
y = y[:200000,:]

In [13]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Note that the entire dataset is used for training

In [14]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
200000/200000 [==============================] - 522s 3ms/step - loss: 2.8729
Epoch 2/20
200000/200000 [==============================] - 553s 3ms/step - loss: 2.5480
Epoch 3/20
200000/200000 [==============================] - 581s 3ms/step - loss: 2.4234
Epoch 4/20
200000/200000 [==============================] - 582s 3ms/step - loss: 2.3348
Epoch 5/20
200000/200000 [==============================] - 544s 3ms/step - loss: 2.2630
Epoch 6/20
200000/200000 [==============================] - 566s 3ms/step - loss: 2.2026
Epoch 7/20
200000/200000 [==============================] - 551s 3ms/step - loss: 2.1519
Epoch 8/20
200000/200000 [==============================] - 549s 3ms/step - loss: 2.1066
Epoch 9/20
200000/200000 [==============================] - 544s 3ms/step - loss: 2.0620
Epoch 10/20
200000/200000 [==============================] - 552s 3ms/step - loss: 2.0222
Epoch 11/20
200000/200000 [==============================] - 542s 3ms/step - loss: 1.9862
Epoch 12/20
200000/

In [15]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [16]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."

Seed:
" that man whom he hath ordained; whereof he hath given assurance
unto all men, in that he hath raised "
 and thet 

4:1 And the LORD Gad said unto him, Th thall be the lat of the erounn  and tooo the sanens of the land of the land of the eroued.

1:: And the LORD said unto him, We taadl the lan whth the sooee, and thet thol shll thte the sooee of the land of the eroued.

1:: And the LORD said unto him, We taadl the lan whth the sooee, and thet thol shll thte the sooee of the land of the eroued.

1:: And the LORD said unto him, We taadl the lan whth the sooee, and thet thol shll thte the sooee of the land of the eroued.

1:: And the LORD said unto him, We taadl the lan whth the sooee, and thet thol shll thte the sooee of the land of the eroued.

1:: And the LORD said unto him, We taadl the lan whth the sooee, and thet thol shll thte the sooee of the land of the eroued.

1:: And the LORD said unto him, We taadl the lan whth the sooee, and thet thol shll thte the sooee of the land 

In [17]:
# serialize model to JSON
model_json = model.to_json()
with open("models/modelgen.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/modelgen.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
#json_file = open('models/modelgen.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
# load weights into new model
#loaded_model.load_weights("models/modelgen.h5")
#print("Loaded model from disk")

Saved model to disk


In [144]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

numpy.ndarray

In [102]:
model.fit(X, y, epochs=20, batch_size=128)

array(4)

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print "Seed:"
print "\"", ''.join([int_to_char[value] for value in pattern]), "\""
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print "\nDone."